In [2]:
import os
# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [3]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from lm_eval import tasks, evaluator
from dotenv import load_dotenv

In [4]:
# Load the .env file
load_dotenv()
# Access the secret values
token = os.getenv('HF_TOKEN')

In [5]:
# Login to Hugging Face
login(token)

# Define the model name
model_name = "google/gemma-2-9b"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

# Load the model in BFloat16 precision
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, token=True)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/stoyang/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm()
        (post_attention_layernorm): Gemma2RMSNorm()
        (pre_feedforwa

In [8]:
# Test the model with a simple input
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Who are you?"
generated_text = generate_text(prompt)
print(generated_text)

Who are you?

I am a 20 year old student from the Netherlands. I am currently studying at the University of Amsterdam, where I am in my third year of the Bachelor of Arts in Communication and Information Sciences.

What is
